In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import re
# import string
import numpy as np
from unicodedata import normalize as unicode_normalize
from abydos.distance import JaroWinkler
JW=JaroWinkler()


def remove_suffix(astring,suffix):
    astring=str(astring)
    if astring.endswith(suffix):
        end = len(suffix)
        return astring[0:-end]
    return astring

abbrev_dict={'dubbed':'dub'}

def clean_name(phrase: str) -> str:
    mypunc='!"#$%&\'()*+,-:;<=>?@[\\]^_`{|}~/'
    phrase=str(phrase)
    if phrase is not None and len(phrase)>1:
        phrase = re.sub('&', ' and ', phrase)
        phrase = re.sub("[\(\[].*?[\)\]]", "", phrase)
        phrase = unicode_normalize('NFKD', phrase.strip().lower())
        # phrase = remove_suffix(phrase,' the')
        phrase = phrase.translate(str.maketrans(mypunc, ' '*len(mypunc)))
        phrase = ''.join([c for c in phrase if c.isalnum() or c.isspace()])
        # phrase = " ".join([abbrev_dict.get(x,x) for x in phrase.split()])
        for k,v in abbrev_dict.items():
            if ' ' in k:
                phrase=phrase.replace(k, v)
        phrase = " ".join([abbrev_dict.get(x,x) for x in phrase.split()])
        return phrase
    else:
        return phrase    


def match_key(name,prefix='the'):
    name=str(name)
    if name is not None and type(name) == str and len(name)>1:
        name=clean_name(name)
        name=name[name.startswith(prefix) and len(prefix):].strip()
        name=remove_suffix(name,' the')
        # name=re.sub(grp_pat, "", name).strip()
        # for suffix in comp_stop_words:
        #     name=remove_suffix(name,suffix)
        return ''.join(sorted(name.split()))
#             return ' '.join(sorted(set(name.split())))
    else:
        return name
def mname_sim_dask(ldt,rdt,lkey,rkey):
    ldt['N1W']=ldt[lkey].str.split(n=1).str.get(0)
    gudict=dict(zip(ldt.N1W.tolist(),ldt[lkey].tolist()))
    gudict={k: v for k, v in gudict.items() if len(k) >2}
    mlst=[]  
    for gu,val in gudict.items():
        if len(gu)>2:
            srch=rdt[rdt[rkey].str.startswith(gu)]
#             srch=rdt[rdt[rkey].str.contains(gu)]
            if len(srch.index) > 0:
                srch[lkey]=val
                mlst.append(srch)

    mdt1=pd.concat(mlst)
    mdt1=mdt1.assign(NAME_SIMILARITY_SCORE=mdt1[[lkey,rkey]].apply(lambda x: 0 if any(x.isnull()) else JW.sim(str(x[0]),str(x[1])), axis=1))
    mdt1.sort_values([lkey,'NAME_SIMILARITY_SCORE'],ascending=[True,False],inplace=True)
    mdt1.drop_duplicates(subset=[lkey],inplace=True)
    return mdt1

In [2]:
# df=pd.read_csv('/Users/deepak.mishra/mywork/csp/data/raw/Series_combine.csv')
df=pd.read_csv('/Users/deepak.mishra/mywork/csp/data/raw/Movie_combine.csv')
df['title_clean']=df['title'].apply(clean_name)
df.sample(10)

,title,file,sheet_name,title_clean
3488,Mausam 2011,Foreign Dubbed Movie List 24-05-2022.xlsx,Bahasa,mausam 2011
3653,Saand Ki Aankh,Foreign Dubbed Movie List 24-05-2022.xlsx,English,saand ki aankh
4600,Mere Jeevan Saathi,Foreign Dubbed Movie List 24-05-2022.xlsx,Russian,mere jeevan saathi
3358,KYUN HO GAYA NA,Foreign Dubbed Movie List 24-05-2022.xlsx,Arabic,kyun ho gaya na
1225,Love in Shimla,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,love in shimla
4824,Aakrosh,Foreign Dubbed Movie List 24-05-2022.xlsx,Tagalog,aakrosh
1694,OK OK (dubbed from Tamil movie-Oru Kal Oru Kan...,Movie_Syndication Movie master jun 2022 for Ba...,Telugu,ok ok
2358,ANTARATAMA,Movie_Syndication Movie master jun 2022 for Ba...,Bangla,antaratama
2439,BIJAYA,Movie_Syndication Movie master jun 2022 for Ba...,Bangla,bijaya
4192,Saheb Biwi Aur Gangster 3,Foreign Dubbed Movie List 24-05-2022.xlsx,Russian,saheb biwi aur gangster 3


In [4]:
#Separate Hindi Title data
dfh=df[df.sheet_name=='Hindi']
dfh.rename(columns={'title_clean':'title_hindi','title':'orig_hindi_title'},inplace=True)
dfh.head()

# df.groupby('sheet_name').agg({'title':'nunique','title_clean':'nunique'}).reset_index()
dfh_df   = pd.merge(dfh,df,how='left',left_on='title_hindi',right_on='title_clean')
dfh_df=dfh_df.assign(NAME_SIMILARITY_SCORE=dfh_df[['title_hindi','title_clean']].apply(lambda x: 0 if any(x.isnull()) else JW.sim(str(x[0]),str(x[1])), axis=1))
dfh_df.sort_values(['title_hindi','NAME_SIMILARITY_SCORE'],ascending=[True,False],inplace=True)
dfh_df.drop(columns=['file_x','sheet_name_x'],inplace=True)
dfh_df.rename(columns={'file_y':'file','sheet_name_y':'sheet_name'},inplace=True)
# dfh_df.drop_duplicates(subset=['title_hindi'],inplace=True)
dfh_df.to_csv('/Users/deepak.mishra/mywork/csp/data/interim/dfh_df.csv',index=False)
dfh_df.head()

,orig_hindi_title,title_hindi,title,file,sheet_name,title_clean,NAME_SIMILARITY_SCORE
358,0.041666666666666664,0041666666666666664,0.041666666666666664,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,0041666666666666664,1.0
292,102 Not Out,102 not out,102 Not Out,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,102 not out,1.0
293,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,Arabic,102 not out,1.0
294,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,English,102 not out,1.0
295,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,German,102 not out,1.0


In [ ]:
dfh_df.columns

mov_sim_dt.columns

dfh.columns

In [5]:
langs=df.sheet_name.unique().tolist()
print(langs)
# dfm=dfh[['title_hindi']]
tmps=[]
for l in langs:
    try:
        tmpdt=mname_sim_dask(dfh,df[df.sheet_name==l],'title_hindi','title_clean')
        dfm=pd.merge(dfh,tmpdt,how='inner',left_on='title_hindi',right_on='title_hindi')
        dfm.drop(columns=['file_x','sheet_name_x','N1W'],inplace=True)
        dfm.rename(columns={'file_y':'file','sheet_name_y':'sheet_name'},inplace=True)
        dfm.to_csv(f'/Users/deepak.mishra/mywork/csp/data/interim/mov_hin_{l}.csv',index=False)
        tmps.append(dfm)
        # tmpdt = tmpdt[['title_hindi','title','NAME_SIMILARITY_SCORE','title_clean']]
        # tmpdt.rename(columns={'title':f'title_{l}'},inplace=True)
        # dfm   = pd.merge(dfm,tmpdt,how='left',left_on='title_hindi',right_on='title_hindi')
        # dfm.rename(columns={'title_clean':f'title_{l}','NAME_SIMILARITY_SCORE':f'SIMSCR_{l}'},inplace=True)

    except:
        continue

mov_sim_dt=pd.concat(tmps)
mov_sim_dt.to_csv('/Users/deepak.mishra/mywork/csp/data/interim/movies_name_sim.csv',index=False)
# dfm.to_csv('/Users/deepak.mishra/mywork/csp/data/interim/movies_name_merged_lang.csv',index=False)


['Hindi', 'Tamil', 'Telugu', 'Kannada', 'Malayalam', 'Bangla', 'Marathi', 'Bhojpuri', 'Punjabi', 'Arabic', 'Bahasa', 'English', 'German', 'Portuguese', 'Spanish', 'Russian', 'Tagalog', 'Thai', 'Vietnamese']


In [ ]:
# dfm=pd.merge(dfh,tmpdt,how='inner',left_on='title_hindi',right_on='title_hindi')
# dfm.drop(columns=['file_x','sheet_name_x','N1W'],inplace=True)
# dfm.rename(columns={'file_y':'file','sheet_name_y':'sheet_name'},inplace=True)
# dfm.head()

# dfm.rename(columns={'title_clean':f'title_{l}','NAME_SIMILARITY_SCORE':f'SIMSCR_{l}'},inplace=True)
# dfm.head()

# tmpdt=mname_sim_dask(dfh,df[df.sheet_name==l],'title_hindi','title_clean')
# tmpdt = tmpdt[['title_hindi','title','NAME_SIMILARITY_SCORE','title_clean']]
# tmpdt = tmpdt.rename(columns={'title':f'title_{l}'},inplace=True)
# tmpdt.head()

In [6]:
mov_sim_dt_dfh_dh=pd.concat([dfh_df,mov_sim_dt])
mov_sim_dt_dfh_dh.drop_duplicates(inplace=True)
mov_sim_dt_dfh_dh.sort_values(by=['title_hindi','sheet_name'],inplace=True)
mov_sim_dt_dfh_dh['Available_Languages']=mov_sim_dt_dfh_dh.groupby('title_hindi')['sheet_name'].transform( lambda x: ', '.join(x))
mov_sim_dt_dfh_dh['Title_in_Languages']=mov_sim_dt_dfh_dh.groupby('title_hindi')['title'].transform( lambda x: ', '.join(x))
mov_sim_dt_dfh_dh.to_csv('/Users/deepak.mishra/mywork/csp/data/interim/mov_sim_dt_dfh_dh.csv',index=False)
mov_sim_dt_dfh_dh.head(20)

,orig_hindi_title,title_hindi,title,file,sheet_name,title_clean,NAME_SIMILARITY_SCORE,Available_Languages,Title_in_Languages
358,0.041666666666666664,0041666666666666664,0.041666666666666664,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,0041666666666666664,1.0,Hindi,0.041666666666666664
293,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,Arabic,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
294,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,English,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
295,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,German,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
292,102 Not Out,102 not out,102 Not Out,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
296,102 Not Out,102 not out,102 NOT OUT,Foreign Dubbed Movie List 24-05-2022.xlsx,Tagalog,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
297,102 Not Out,102 not out,102 Not Out,Foreign Dubbed Movie List 24-05-2022.xlsx,Thai,102 not out,1.0,"Arabic, English, German, Hindi, Tagalog, Thai","102 Not Out, 102 Not Out, 102 Not Out, 102 Not..."
17,14 Phere,14 phere,14 Phere,Movie_Syndication Movie master jun 2022 for Ba...,Hindi,14 phere,1.0,"Hindi, Russian","14 Phere, 14 Phere"
18,14 Phere,14 phere,14 Phere,Foreign Dubbed Movie List 24-05-2022.xlsx,Russian,14 phere,1.0,"Hindi, Russian","14 Phere, 14 Phere"
664,1920 London,1920 london,1920 London,Foreign Dubbed Movie List 24-05-2022.xlsx,Arabic,1920 london,1.0,"Arabic, Bahasa, English, German, Hindi, Spanis...","1920 London, 1920 London, 1920 London, 1920 Lo..."


In [7]:
mov_sim_dt_dfh_dh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3749 entries, 358 to 453
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   orig_hindi_title       3749 non-null   object 
 1   title_hindi            3749 non-null   object 
 2   title                  3749 non-null   object 
 3   file                   3749 non-null   object 
 4   sheet_name             3749 non-null   object 
 5   title_clean            3749 non-null   object 
 6   NAME_SIMILARITY_SCORE  3749 non-null   float64
 7   Available_Languages    3749 non-null   object 
 8   Title_in_Languages     3749 non-null   object 
dtypes: float64(1), object(8)
memory usage: 292.9+ KB


In [8]:

# Creates a pivot table dataframe

title_in_langs=mov_sim_dt_dfh_dh.pivot_table(index=['orig_hindi_title', 'title_hindi'],
                                     columns='sheet_name', 
                                     values='title',
                                     aggfunc=lambda x: ' '.join(x)).reset_index()

title_in_langs.head()

sheet_name,orig_hindi_title,title_hindi,Arabic,Bahasa,Bangla,Bhojpuri,English,German,Hindi,Kannada,...,Marathi,Portuguese,Punjabi,Russian,Spanish,Tagalog,Tamil,Telugu,Thai,Vietnamese
0,0.041666666666666664,0041666666666666664,NaN,NaN,NaN,NaN,NaN,NaN,0.041666666666666664,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102 Not Out,102 not out,102 Not Out,NaN,NaN,NaN,102 Not Out,102 Not Out,102 Not Out,NaN,...,NaN,NaN,NaN,NaN,NaN,102 NOT OUT,NaN,NaN,102 Not Out,NaN
2,14 Phere,14 phere,NaN,NaN,NaN,NaN,NaN,NaN,14 Phere,NaN,...,NaN,NaN,NaN,14 Phere,NaN,NaN,NaN,NaN,NaN,NaN
3,1920 London,1920 london,1920 London,1920 London,NaN,NaN,1920 London,1920 London,1920 London,NaN,...,NaN,NaN,NaN,NaN,1920 London,1920 London,NaN,NaN,1920 London,1920 London
4,2001: Do Hazaar Ek,2001 do hazaar ek,NaN,NaN,NaN,NaN,NaN,NaN,2001: Do Hazaar Ek,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
langs_comb=mov_sim_dt_dfh_dh.groupby('title_hindi').agg({'sheet_name': lambda x: ', '.join(x)}).reset_index()
langs_comb.columns=['title_hindi','Available_Languages']
langs_comb

,title_hindi,Available_Languages
0,0041666666666666664,Hindi
1,102 not out,"Arabic, English, German, Hindi, Tagalog, Thai"
2,14 phere,"Hindi, Russian"
3,1920 london,"Arabic, Bahasa, English, German, Hindi, Spanis..."
4,2001 do hazaar ek,Hindi
...,...,...
1363,zamaana deewana,Hindi
1364,zamana mujh se hai dub,"Hindi, Kannada"
1365,zehreela insaan,Hindi
1366,ziddi,"Bhojpuri, Hindi"


In [10]:
title_in_langs=pd.merge(title_in_langs,langs_comb,how='left',on='title_hindi')
title_in_langs

,orig_hindi_title,title_hindi,Arabic,Bahasa,Bangla,Bhojpuri,English,German,Hindi,Kannada,...,Portuguese,Punjabi,Russian,Spanish,Tagalog,Tamil,Telugu,Thai,Vietnamese,Available_Languages
0,0.041666666666666664,0041666666666666664,NaN,NaN,NaN,NaN,NaN,NaN,0.041666666666666664,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hindi
1,102 Not Out,102 not out,102 Not Out,NaN,NaN,NaN,102 Not Out,102 Not Out,102 Not Out,NaN,...,NaN,NaN,NaN,NaN,102 NOT OUT,NaN,NaN,102 Not Out,NaN,"Arabic, English, German, Hindi, Tagalog, Thai"
2,14 Phere,14 phere,NaN,NaN,NaN,NaN,NaN,NaN,14 Phere,NaN,...,NaN,NaN,14 Phere,NaN,NaN,NaN,NaN,NaN,NaN,"Hindi, Russian"
3,1920 London,1920 london,1920 London,1920 London,NaN,NaN,1920 London,1920 London,1920 London,NaN,...,NaN,NaN,NaN,1920 London,1920 London,NaN,NaN,1920 London,1920 London,"Arabic, Bahasa, English, German, Hindi, Spanis..."
4,2001: Do Hazaar Ek,2001 do hazaar ek,NaN,NaN,NaN,NaN,NaN,NaN,2001: Do Hazaar Ek,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hindi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369,Zamana Mujh Se Hai-dubbed,zamana mujh se hai dub,NaN,NaN,NaN,NaN,NaN,NaN,Zamana Mujh Se Hai-dubbed,Zamana,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Hindi, Kannada"
1370,Zehreela Insaan,zehreela insaan,NaN,NaN,NaN,NaN,NaN,NaN,Zehreela Insaan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hindi
1371,Ziddi,ziddi,NaN,NaN,NaN,ziddi ashiq,NaN,NaN,Ziddi,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bhojpuri, Hindi"
1372,Zinda Hoon Main - Dub,zinda hoon main dub,NaN,NaN,NaN,NaN,NaN,NaN,Zinda Hoon Main - Dub,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Hindi


In [11]:
title_in_langs.to_csv('/Users/deepak.mishra/mywork/csp/data/interim/title_in_langs.csv',index=False)

In [ ]:
# pip install pandas-dedupe
# import pandas as pd
# import pandas_dedupe

# #load dataframe
# # df = pd.read_csv('test_names.csv')

# #initiate deduplication
# df_final = pandas_dedupe.dedupe_dataframe(df,['title', 'title_clean'])

# #send output to csv
# # df_final.to_csv('deduplication_output.csv')

# df_final.nunique()

# df_final.to_csv('deduplication_output.csv')

# import os
# os.getcwd()

# import pandas as pd
# import pandas_dedupe

# #load dataframes
# dfa = df
# dfb = df

# #initiate matching
# df_final = pandas_dedupe.link_dataframes(dfa, dfb, ['title_clean','title'])

# #send output to csv
# df_final.to_csv('linkage_output.csv')

# df.file.unique()

# df.columns